## SQL-запросы для анализа ключевых бизнес метрик сервиса доставки еды.

Для анализа предоставлены 5 таблиц с данными сервиса доставки еды:

* журнал, содержащий логи событий,
* информация о рекламных затратах по дням и каналам привлечения,
* справочник партнёрских сетей и их ресторанов,
* справочник блюд,
* справочник городов.

### Задача 1. Расчёт DAU

In [ ]:
SELECT log_date,
       COUNT (DISTINCT user_id) AS DAU
FROM analytics_events
JOIN cities USING (city_id)
WHERE log_date BETWEEN '2021-05-01' AND '2021-06-30' AND city_name = 'Саранск' AND event = 'order'
GROUP BY log_date
ORDER BY log_date;

### Задача 2. Расчёт Conversion Rate

In [ ]:
SELECT log_date,
       ROUND(COUNT(DISTINCT user_id) filter (WHERE event = 'order')::numeric/ COUNT(DISTINCT user_id), 2) AS CR
FROM analytics_events
JOIN cities USING (city_id)
WHERE log_date BETWEEN '2021-05-01' AND '2021-06-30' AND city_name = 'Саранск'
GROUP BY log_date
ORDER BY log_date;

### Задача 3. Расчёт среднего чека

In [ ]:
-- Рассчитываем величину комиссии с каждого заказа, отбираем заказы по дате и городу
WITH orders AS
    (SELECT *,
            revenue * commission AS commission_revenue
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE revenue IS NOT NULL
         AND log_date BETWEEN '2021-05-01' AND '2021-06-30'
         AND city_name = 'Саранск')

SELECT DATE_TRUNC('month', log_date)::date AS "Месяц",
       COUNT (DISTINCT order_id) filter (WHERE event = 'order') AS "Количество заказов",
       ROUND(SUM (commission_revenue) filter (WHERE event = 'order')::numeric,2) AS "Сумма комиссии",
       ROUND(SUM (commission_revenue) filter (WHERE event = 'order')::numeric / COUNT (DISTINCT order_id) filter (WHERE event = 'order'), 2) AS "Средний чек"
FROM orders
GROUP BY DATE_TRUNC('month', log_date)
ORDER BY "Месяц";

### Задача 4. Расчёт LTV ресторанов

In [ ]:
-- Рассчитываем величину комиссии с каждого заказа, отбираем заказы по дате и городу
WITH orders AS
    (SELECT analytics_events.rest_id,
            analytics_events.city_id,
            revenue * commission AS commission_revenue
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE revenue IS NOT NULL
         AND log_date BETWEEN '2021-05-01' AND '2021-06-30'
         AND city_name = 'Саранск')

SELECT o.rest_id,
       p.chain AS "Название сети",
       p.type AS "Тип кухни",
       ROUND(CAST(SUM (commission_revenue) AS numeric), 2) AS LTV
FROM orders AS o
LEFT JOIN partners AS p USING (rest_id) 
WHERE p.city_id = 6
GROUP BY rest_id, chain, type
ORDER BY LTV DESC;

### Задача 5. Расчёт LTV ресторанов — самые популярные блюда

In [ ]:
-- Рассчитываем величину комиссии с каждого заказа, отбираем заказы по дате и городу
WITH orders AS
    (SELECT analytics_events.rest_id,
            analytics_events.city_id,
            analytics_events.object_id,
            revenue * commission AS commission_revenue
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE revenue IS NOT NULL
         AND log_date BETWEEN '2021-05-01' AND '2021-06-30'
         AND city_name = 'Саранск'), 

-- Рассчитываем два ресторана с наибольшим LTV 
top_ltv_restaurants AS
    (SELECT orders.rest_id,
            chain,
            type,
            ROUND(SUM(commission_revenue)::numeric, 2) AS LTV
     FROM orders
     JOIN partners ON orders.rest_id = partners.rest_id AND orders.city_id = partners.city_id
     GROUP BY 1, 2, 3
     ORDER BY LTV DESC
     LIMIT 2)

SELECT t.chain AS "Название сети",
       d.name AS "Название блюда",
       d.spicy,
       d.fish,
       d.meat,
       ROUND(SUM (o.commission_revenue)::numeric, 2) AS LTV
FROM dishes as d
JOIN top_ltv_restaurants AS t USING (rest_id)
JOIN orders AS o USING (object_id)
GROUP BY t.chain, d.name, d.spicy, d.fish, d.meat
ORDER BY LTV DESC;



### Задача 6. Расчёт Retention Rate

In [ ]:
-- Рассчитываем новых пользователей по дате первого посещения продукта
WITH new_users AS
    (SELECT DISTINCT first_date,
                     user_id
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE first_date BETWEEN '2021-05-01' AND '2021-06-24'
         AND city_name = 'Саранск'),

-- Рассчитываем активных пользователей по дате события
active_users AS
    (SELECT DISTINCT log_date,
                     user_id
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE log_date BETWEEN '2021-05-01' AND '2021-06-30'
         AND city_name = 'Саранск'),

-- Выделим первую неделю по дням
daily_retention AS
     (SELECT new_users.user_id,
             first_date,
             log_date::date - first_date::date AS day_since_install
     FROM new_users
     JOIN active_users ON new_users.user_id = active_users.user_id
     AND log_date >= first_date)
SELECT day_since_install,
       COUNT (DISTINCT user_id) AS retained_users,
       ROUND((1.0 * COUNT(DISTINCT user_id) / MAX(COUNT(DISTINCT user_id)) OVER (ORDER BY day_since_install))::numeric, 2) AS retention_rate
FROM daily_retention
WHERE day_since_install < 8
GROUP BY day_since_install
ORDER BY day_since_install;

### Задача 7. Сравнение Retention Rate по месяцам

In [ ]:
-- Рассчитываем новых пользователей по дате первого посещения продукта
WITH new_users AS
    (SELECT DISTINCT first_date,
                     user_id
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE first_date BETWEEN '2021-05-01' AND '2021-06-24'
         AND city_name = 'Саранск'),

-- Рассчитываем активных пользователей по дате события
active_users AS
    (SELECT DISTINCT log_date,
                     user_id
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE log_date BETWEEN '2021-05-01' AND '2021-06-30'
         AND city_name = 'Саранск'),

-- Соединяем таблицы с новыми и активными пользователями
daily_retention AS
    (SELECT new_users.user_id,
            first_date,
            log_date::date - first_date::date AS day_since_install
     FROM new_users
     JOIN active_users ON new_users.user_id = active_users.user_id
     AND log_date >= first_date)

SELECT DATE_TRUNC('month', first_date)::date AS "Месяц",
       day_since_install,
       COUNT (DISTINCT user_id) AS retained_users,
       ROUND((1.0 * COUNT(DISTINCT user_id) / MAX(COUNT(DISTINCT user_id)) OVER (PARTITION BY DATE_TRUNC('month', first_date)::date ORDER BY day_since_install))::numeric, 2) AS retention_rate
FROM daily_retention
WHERE day_since_install < 8
GROUP BY "Месяц", day_since_install
ORDER BY "Месяц", day_since_install;